# Scale-Dependent Bias Modification

Initiate notebook.

In [1]:
import os
import pprint
from collections import OrderedDict

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.lines import Line2D
from nbodykit.lab import cosmology
from scipy.integrate import quad
from scipy.misc import derivative

In [2]:
FIDUCIAL_COSMOLOGY = cosmology.Planck15

## Base cosmological model

Define the scale-dependence modification to bias per unit primordial non-Gaussianity per unit Lagrangian bias.

In [3]:
def scale_dependence_modification(redshift, cosmo=FIDUCIAL_COSMOLOGY):
    """Return the scale-dependence modification as a function of redshift.

    Parameters
    ----------
    redshift : float
        Redshift.
    cosmo : :class:`nbodykit.cosmology.Cosmology`, optional
        Base cosmological model (default is
        :obj:`nbodykit.cosmology.Planck15`).

    Returns
    -------
    callable
        Scale-dependence modification as a function of the Fourier scale
        (in h/Mpc).

    """
    SPHERICAL_COLLAPSE_CRITICAL_OVERDENSITY = 1.686
    SPEED_OF_LIGHT_IN_HUNDRED_KM_PER_S = 2998.

    num_factors = 3 * (cosmo.h / SPEED_OF_LIGHT_IN_HUNDRED_KM_PER_S)**2\
        * SPHERICAL_COLLAPSE_CRITICAL_OVERDENSITY * cosmo.Omega0_m
    transfer_func = cosmology.power.transfers.CLASS(cosmo, redshift)

    return lambda k: num_factors / (k**2 * transfer_func(k))

Define the general relativistic corrections as a function of redshift due to evolution and magnification of intrinsic
background tracer number density.

In [4]:
def general_relativistic_corrections(evolution_bias, magnification_bias,
                                     cosmo=FIDUCIAL_COSMOLOGY):
    """Return the general relativistic corrections as a function of 
    redshift.

    Parameters
    ----------
    evolution_bias, magnification_bias : callable
        Evolution bias or magnification bias as a function of redshift.
    cosmo : :class:`nbodykit.cosmology.Cosmology`, optional
        Base cosmological model (default is
        ``nbodykit.cosmology.Planck15``).

    Returns
    -------
    callable
        General relativistic correction as a function of redshift.

    """
    background_evolution = cosmology.background.MatterDominated(cosmo.Omega0_m)
    
    hubble_parameter = lambda z: 1/(1+z) * background_evolution.E(1/(1+z))
    
    pass

## Luminosty function modeller

Generate parametric models for the luminosity function and perform related computations.

In [5]:
class LuminosityFunctionModel:
    """Parametric luminosity function model predicting the comoving number
    density and related quantities.
    
    Parameters
    ----------
    luminosity_function : callable
        Luminosity function of magnitude and redshift (in that order).
    magnitude_limit : float
        Limiting magnitude.
    **model_parameters
        Keyword arguments to be passed to the specified parametric model.
    
    Attributes
    ----------
    luminosity_function : callable
        Luminosity function of magnitude and redshift (in that order).
    magnitude_limit : float
        Limiting magnitude.
    model_parameters : dict
        Model parameters.
    
    """
    
    def __init__(self, luminosity_function, magnitude_limit, 
                 **model_parameters):
            
        self.luminosity_function = luminosity_function
        self.magnitude_limit = magnitude_limit
        self.model_parameters = model_parameters
            
        self._comoving_number_density = None
        self._evolution_bias = None
        self._magnification_bias = None
        
    @classmethod
    def from_parameters_file(cls, file_path, luminosity_function, 
                             magnitude_limit):
        """Instantiate a parametric model by reading parameter values from 
        a file.
        
        Parameters
        ----------
        file_path : str
            Path of the model parameter file.
        luminosity_function : callable
            Luminosity function of magnitude and redshift (in that order).
        magnitude_limit : float
            Limiting magnitude.
                
        """
        with open(file_path, 'r') as pfile:
            parameters = tuple(map(
                lambda var_name: var_name.strip(" "), 
                pfile.readline().strip("#").strip("\n").split(",")
            ))
            estimates = tuple(map(
                lambda value: float(value), 
                pfile.readline().split(",")
            ))
            
        return cls(
            luminosity_function, magnitude_limit, 
            **dict(zip(parameters, estimates))
        )
    
    @property
    def comoving_number_density(self):
        """Comoving number density as a function of redshift.
        
        Returns
        -------
        callable
        
        """
        if callable(self._comoving_number_density):
            return self._comoving_number_density
        
        self._comoving_number_density = lambda z: quad(
            self.luminosity_function,
            -np.inf, 
            self.magnitude_limit,
            args=(z,)
        )
        
        return self._comoving_number_density
    
    @property
    def evolution_bias(self):
        """Evolution bias as a function of redshift.
        
        Returns
        -------
        callable
        
        """
        if callable(self._evolution_bias):
            return self._evolution_bias
        
        self._evolution_bias = lambda z: -(1 + z) \
            * derivative(self.comoving_number_density, z, dx=0.05)\
            / self.comoving_number_density(z)
        
        return self._evolution_bias
    
    @property
    def magnification_bias(self):
        """Magnification bias as a function of redshift.
        
        Returns
        -------
        callable
        
        """        
        if callable(self._magnification_bias):
            return self._magnification_bias
        
        self._magnification_bias = lambda z:\
            self.luminosity_function(self.magnitude_limit, z)\
            / self.comoving_number_density(z)
        
        return self._magnification_bias

## General relativistic modification

Specify the luminosity function model.

In [6]:
def quasar_luminosity_function_in_PLE_model(magnitude, redshift, 
                                            redshift_pivot=2.2, 
                                            **model_parameters_PLE):
    """Evaluate the pure luminosity evolution (PLE) model for the quasar
    luminosity function at the given absolute magnitude and redshift.
    
    Notes
    -----
    Magnitude is absolute and measured in :math:`g`-band normalised to
    the value at redshift 2.
    
    Parameters
    ----------
    magnitude : float
        Quasar magnitude.
    redshift : float
        Quasar redshift.
    redshift_pivot : float, optional
        Pivot redshift.
    **model_parameters_PLE
        PLE model parameters.
    
    Returns
    -------
    comoving_density : float :class:`numpy.ndarray`
        Predicted qausar comoving number density per unit magnitude.
        
    """    
    # Re-definitions.
    M_g, z, z_p = magnitude, redshift, redshift_pivot
    
    # Determine the redshift end.
    if z <= z_p:
        subscript = '\\textrm{{{}}}'.format('l')
    else:
        subscript = '\\textrm{{{}}}'.format('h') 
    
    # Set parameters.
    Phi_star = 10**model_parameters_PLE['\\log\\Phi^\\ast']
    M_g_star_p = model_parameters_PLE['M^\\ast_g(z_\\textrm{pivot})']
    
    alpha = model_parameters_PLE['\\alpha_{}'.format(subscript)]
    beta = model_parameters_PLE['\\beta_{}'.format(subscript)]
    k_1 = model_parameters_PLE['k_{{1{}}}'.format(subscript)]
    k_2 = model_parameters_PLE['k_{{2{}}}'.format(subscript)]
    
    # Evaluate the model prediction.
    exponent_magnitude_factor = M_g \
        - (M_g_star_p - 2.5*(k_1 * (z - z_p) + k_2 * (z - z_p)**2))
    faint_power_law = 10 ** (0.4*(alpha + 1) * exponent_magnitude_factor)
    bright_power_law = 10 ** (0.4*(beta + 1) * exponent_magnitude_factor)
    
    comoving_density = Phi_star / (faint_power_law + bright_power_law)
    
    return comoving_density

Specify fiducial redshift, matter power spectrum and growth rate.

In [7]:
FIDUCIAL_REDSHIFT = 2.

matter_power_spectrum = cosmology.LinearPower(
    FIDUCIAL_COSMOLOGY, FIDUCIAL_REDSHIFT
)
growth_rate = \
    FIDUCIAL_COSMOLOGY.scale_independent_growth_rate(FIDUCIAL_REDSHIFT)